In [1]:
MODEL_NAME = 'first_try'

In [2]:
import pandas as pd
import os

def get_image_path(image_id:int):
    return os.path.join('../tiles_768', str(image_id))

train = pd.read_csv(f"../data/train.csv")

train['tile_path'] = train['image_id'].apply(lambda x: get_image_path(x))
train.head()

,image_id,label,image_width,image_height,is_tma,tile_path
0,4,HGSC,23785,20008,False,../tiles_768/4
1,66,LGSC,48871,48195,False,../tiles_768/66
2,91,HGSC,3388,3388,True,../tiles_768/91
3,281,LGSC,42309,15545,False,../tiles_768/281
4,286,EC,37204,30020,False,../tiles_768/286


In [3]:
from PIL import Image
import torch
import torch.nn as nn
import timm
from timm.models.layers import DropPath
import copy
from itertools import cycle

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "timm/tiny_vit_21m_224.dist_in22k_ft_in1k"

print(f"Using device {device} and model {model_name}")

model = timm.create_model(model_name, pretrained=True)

# drop_path_rate = 0.1
dropout_rate = 0.1
# drop_path_rates = []
# total_blocks = 0
# for stage in model.stages:
#     if hasattr(stage, 'blocks'):
#         for block in stage.blocks:
#             total_blocks += 1
# drop_path_rates = [x.item() for x in torch.linspace(0, drop_path_rate, total_blocks)]
        
i_drop = 0
# Assign drop path rates
for stage in model.stages:
    if hasattr(stage, 'blocks'):
        for block in stage.blocks:
            # block.drop_path = DropPath(drop_prob=drop_path_rates[i_drop])
            # block.drop_path1 = DropPath(drop_prob=drop_path_rates[i_drop])
            # block.drop_path2 = DropPath(drop_prob=drop_path_rates[i_drop])
            if hasattr(block, 'mlp'):
                block.mlp.drop1 = nn.Dropout(p=dropout_rate, inplace=False)
                block.mlp.drop2 = nn.Dropout(p=dropout_rate, inplace=False)
            
            i_drop += 1

model.head.drop = nn.Dropout(p=dropout_rate, inplace=False)
model.head.fc = nn.Linear(model.head.in_features, 5)
state_dict = torch.load('tinyvit_models_first_try/epoch_0_step_4000.pth', map_location=device)
model.load_state_dict(state_dict, strict=False)
model = model.to(device)

# Initialize EMA model
ema_decay = 0.999  # decay factor for EMA
ema_model = copy.deepcopy(model)
ema_model = ema_model.to(device)

Using device cuda and model timm/tiny_vit_21m_224.dist_in22k_ft_in1k


In [4]:
import os
from PIL import Image
from torch.utils.data import Dataset
import random

integer_to_label = {
    0: 'HGSC',
    1: 'CC',
    2: 'EC',
    3: 'LGSC',
    4: 'MC',
}

label_to_integer = {
    'HGSC': 0,
    'CC': 1,
    'EC': 2,
    'LGSC': 3,
    'MC': 4,
}
    
class ImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.all_images = []  # Store all images in an interlaced fashion
        self.folder_images = []  # Temporary storage for images from each folder

        # Step 1: Collect all images from each folder
        for index, row in dataframe.iterrows():
            folder_path = row['tile_path']
            label = row['label']
            image_id = row['image_id']
            if os.path.isdir(folder_path):
                image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith('.png')]
                random.shuffle(image_files)
                self.folder_images.append([(image_file, label, image_id) for image_file in image_files])

        # Step 2: Interlace the images
        max_length = max(len(images) for images in self.folder_images)
        for i in range(max_length):
            for images in self.folder_images:
                self.all_images.append(images[i % len(images)])

    def __len__(self):
        return len(self.all_images)

    def __getitem__(self, idx):
        image_path, label, image_id = self.all_images[idx]
        image = Image.open(image_path)
        
        if self.transform:
            image = self.transform(image)

        return image, label_to_integer[label], image_id

In [5]:
from torch.utils.data import DataLoader, WeightedRandomSampler
import torchvision.transforms as transforms

BATCH_SIZE = 64

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(size=224, scale=(0.75, 1.0), ratio=(0.75, 1.33)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=(0, 360)),
    transforms.RandomAffine(degrees=0, shear=(-20, 20, -20, 20)),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.3, hue=0.3),
    transforms.RandomApply([transforms.Grayscale(num_output_channels=3)], p=0.25),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 1))], p=0.25),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[
        0.485,
        0.456,
        0.406
    ], std=[
        0.229,
        0.224,
        0.225
    ]),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False),
])

val_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[
        0.485,
        0.456,
        0.406
    ], std=[
        0.229,
        0.224,
        0.225
    ]),
])

train_dataset = ImageDataset(dataframe=train, transform=train_transform)

# Calculate weights for each class
class_counts = [train.groupby('label').count().loc[label]['image_id'] for label in label_to_integer]  # Example class counts
num_samples = sum(class_counts)
class_weights = [num_samples / class_count for class_count in class_counts]

# Assign a weight to each sample in the dataset based on its class
sample_weights = [class_weights[label_to_integer[label]] for _, label, _ in train_dataset.all_images]

# Create WeightedRandomSampler
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

# DataLoader with WeightedRandomSampler
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler, num_workers=3)

In [6]:
import logging
import sys

# Get the root logger
logger = logging.getLogger()

# Optional: Remove all existing handlers from the logger
for handler in logger.handlers[:]:
    logger.removeHandler(handler)

# Set the logging level
logger.setLevel(logging.INFO)

# Create a FileHandler and add it to the logger
file_handler = logging.FileHandler(f'logs/tinyvit_train_{MODEL_NAME}.txt')
file_handler.setLevel(logging.INFO)
logger.addHandler(file_handler)

# Create a StreamHandler for stderr and add it to the logger
stream_handler = logging.StreamHandler(sys.stderr)
stream_handler.setLevel(logging.ERROR)  # Only log ERROR and CRITICAL messages to stderr
logger.addHandler(stream_handler)

In [ ]:
import torch
import torch.optim as optim
import logging
import numpy as np
import math
from sklearn.metrics import balanced_accuracy_score
import random
from torch.cuda.amp import GradScaler, autocast

initial_lr = 0.0005 * BATCH_SIZE/1024
final_lr = initial_lr * 0.01
num_epochs = 10000

# Function for linear warmup
def learning_rate(step, warmup_steps=1000, max_steps=10000):
    if step < warmup_steps:
        return initial_lr * (float(step) / float(max(1, warmup_steps)))
    elif step < max_steps:
        progress = (float(step - warmup_steps) / float(max(1, max_steps - warmup_steps)))
        cos_component = 0.5 * (1 + math.cos(math.pi * progress))
        return final_lr + (initial_lr - final_lr) * cos_component
    else:
        return final_lr

def update_ema_variables(model, ema_model, alpha):
    # Update the EMA model parameters
    with torch.no_grad():
        for ema_param, param in zip(ema_model.parameters(), model.parameters()):
            ema_param.data.mul_(alpha).add_(param.data, alpha=1 - alpha)

scaler = GradScaler()
optimizer = optim.AdamW(model.parameters(), lr=initial_lr, weight_decay=1e-8)

# Define the loss function with class weights
criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.1)

best_val_accuracy = 0.0
step = 0

for epoch in range(num_epochs):
    model.train()  # set the model to training mode
    
    for i, (images, labels, _) in enumerate(train_dataloader, 0):
        # Convert images to PIL format
        images = images.to(device)
        labels = labels.to(device)
        
        # Linearly increase the learning rate
        lr = learning_rate(step)
        for g in optimizer.param_groups:
            g['lr'] = lr

        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass with autocast
        with autocast():
            outputs = model(images)
            logits_per_image = outputs
            loss = criterion(logits_per_image, labels)
        
        # Backward pass
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        update_ema_variables(model, ema_model, ema_decay)

        logging.info('[%d, %5d] loss: %.3f' % (epoch + 1, step, loss.item()))

        if step % 500 == 0:
            ema_model.eval()
            torch.save(ema_model.state_dict(), f'tinyvit_models_{MODEL_NAME}/epoch_{epoch}_step_{step}.pth')
            logging.info(f'Model saved after epoch {epoch} and step {step}')\

            model.train()

        if step == 10000:
            torch.save(ema_model.state_dict(), f'tinyvit_models_{MODEL_NAME}/final.pth')
            break
    
        step += 1
        
    if step >= 10000:
        break

In [ ]:
import shutil
import os

def duplicate_ipynb_with_new_name(src_file_path, dest_dir, new_name):
    """
    Duplicate an IPython notebook file to a new directory with a new file name.

    Parameters:
    src_file_path (str): The path of the source IPython notebook file.
    dest_dir (str): The destination directory where the file will be copied.
    new_name (str): The new file name for the duplicated notebook.

    Returns:
    str: The path of the duplicated file with the new name.
    """
    # Check if the new name contains the '.ipynb' extension, add if not
    if not new_name.endswith('.ipynb'):
        new_name += '.ipynb'

    # Creating the destination file path with the new name
    dest_file_path = os.path.join(dest_dir, new_name)

    # Copying the file to the new directory
    shutil.copy(src_file_path, dest_file_path)

    return dest_file_path

src_file = "eva02-base-finetune.ipynb"
dest_directory = "notebook_history"
new_filename = f"{MODEL_NAME}.ipynb"
duplicate_ipynb_with_new_name(src_file, dest_directory, new_filename)


In [6]:
import random
from sklearn.metrics import balanced_accuracy_score

ema_model.eval()

image_ids = []
logits = []
labels = []

with torch.no_grad():
    for _, row in train.iterrows():
        path = row['tile_path']
        all_files = [f for f in os.listdir(path) if f.lower().endswith('.png')]

        sum_probabilities = torch.zeros(5).to(device)
        sum_log_probabilities = torch.zeros(5).to(device)
        sum_log_neg_probabilities = torch.zeros(5).to(device)
        
        batch_logits = []

        # Prepare a list to hold image tiles
        batch_tiles = []

        sample_size = min(32, len(all_files))
        sampled_files = random.sample(all_files, sample_size)

        for image_name in sampled_files:
            image_path = os.path.join(path, image_name)
            sub_image = Image.open(image_path)

            tile = val_transform(sub_image).unsqueeze(0).to(device)
            batch_tiles.append(tile)
        
        for i_batch in range(0, len(batch_tiles), 32):
            outputs = ema_model(torch.concat(batch_tiles[i_batch:i_batch+32], dim=0))
            probs = outputs.softmax(dim=1)
            batch_logits.append(outputs)
            sum_probabilities += probs.sum(dim=0)
            sum_log_probabilities += torch.log(probs).sum(dim=0)
            sum_log_neg_probabilities += torch.log(1 - probs).sum(dim=0)
        
        image_id = row['image_id']
        batch_logits = torch.concat(batch_logits, dim=0)
        mean_logits = batch_logits.mean(dim=0)
        label = row['label']
        print(batch_logits.shape, label, image_id, integer_to_label[mean_logits.argmax().cpu().item()], mean_logits)
        image_ids.append(image_id)
        logits.append(batch_logits)
        labels.append(label)
        

torch.Size([32, 5]) HGSC 4 HGSC tensor([ 0.3912,  0.1024,  0.2623,  0.1081, -0.8786], device='cuda:0')
torch.Size([32, 5]) LGSC 66 HGSC tensor([ 0.3081,  0.1374,  0.2456,  0.1123, -0.6771], device='cuda:0')
torch.Size([18, 5]) HGSC 91 HGSC tensor([ 0.9344, -0.3207,  0.2798, -0.0508, -0.6196], device='cuda:0')
torch.Size([32, 5]) LGSC 281 EC tensor([ 0.3054, -0.0984,  0.3649, -0.1066, -0.3939], device='cuda:0')
torch.Size([32, 5]) EC 286 EC tensor([ 0.4677, -0.3240,  1.1658, -0.6466, -0.2813], device='cuda:0')
torch.Size([32, 5]) HGSC 431 CC tensor([ 0.2733,  0.3486,  0.1957, -0.3043, -0.5536], device='cuda:0')
torch.Size([32, 5]) HGSC 706 HGSC tensor([ 0.5510, -0.3401,  0.4300, -0.3161, -0.1999], device='cuda:0')
torch.Size([32, 5]) HGSC 970 EC tensor([ 0.0111,  0.0716,  0.2086,  0.1667, -0.4613], device='cuda:0')
torch.Size([32, 5]) HGSC 1020 EC tensor([ 0.5111,  0.0762,  0.6344, -0.3561, -0.8774], device='cuda:0')
torch.Size([32, 5]) HGSC 1080 EC tensor([ 0.4158, -0.0538,  0.5244, -0


KeyboardInterrupt



In [7]:
predictions = []
for image_logits in logits:
    summed_logits = image_logits.sum(dim=0)
    
    max_vote_key = summed_logits.argmax().cpu().item()
    predictions.append(integer_to_label[max_vote_key])

logit_sum_accuracy = balanced_accuracy_score(labels, predictions)
logit_sum_accuracy

0.7246153846153847